In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import time
import json
import csv
import os
import pandas as pd
from tqdm import tqdm_notebook
from urllib.error import URLError
import time

HEADERS = {'User-Agent': 'Mozilla/5.0', 'Accept': 'text/html,application/xhtml+xml,application/xml'}

In [2]:
f = open("./scraped-data/UE-categories-popular.txt")
categories = f.read().split('\n')
if '' in categories:
    categories.remove('')
len(categories)

52

In [3]:
def get_city_restaurants(city_url, categories, headers=HEADERS):
    restaurant_set = set()
    cityname = city_url.split('/')[-1]
    
    # checking if city exists
    try:
        req = Request(city_url, headers=headers)
        webpage = urlopen(req, timeout=20).read()
        soup = BeautifulSoup(webpage, 'html.parser')
    except Exception as e:
        print(f"Error: {e}, url: {city_url}")
        return rests, categ_stats
    
    # now fetching all restaurants urls category-wise, and storing urls in restaurant_set
    for category in tqdm_notebook(categories, desc=cityname):
        category = category.lower().replace(" ", "-")
        category_city_url = "https://www.ubereats.com/category/"+cityname+"/"+category
        try:
            # fetching restaurants in a category
            req = Request(category_city_url, headers=headers)
            webpage = urlopen(req, timeout=20).read()
            soup = BeautifulSoup(webpage, 'html.parser')
            links = soup.find_all("a")
            links = [link['href'] for link in links 
                     if len(link['href'].split('/'))>=3 
                     and link['href'].split('/')[2] == 'food-delivery']
            
            # adding link to the final city restaurant_set
            for link in links:
                restaurant_set.add(link)   
        except Exception as e:
           print(f"\nerror: {e}\n url: {category_city_url}")
        
    return restaurant_set

In [4]:
def store_urls(city_url, city_restaurants):
    cityname = city_url.split('/')[-1]
    filename = './scraped-data/restaurant-urls/'+cityname+'.csv'
    city_df = pd.DataFrame(columns=['url', 'city'])
    
    i=0
    for rest_url in city_rests:
        city_df.loc[i] = [
            "https://www.ubereats.com" + rest_url,
            cityname
        ]
        i = i+1
    city_df.to_csv(filename, index=False)

In [6]:
city_url = "https://www.ubereats.com/location/new-york"
city_restaurants = get_city_restaurants(city_url, categories)
store_urls(city_url, city_restaurants)

KeyboardInterrupt: 